# Working with Data Structures and Schemas in Gemini Function Calling


## Overview

Gemini is a family of generative AI models developed by Google DeepMind that is designed for multimodal use cases. The Gemini API gives you access to the Gemini models.

[Function Calling](https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling) in Gemini lets you create a description of a function in your code, then pass that description to a language model in a request. The response from the model includes the name of a function that matches the description and the arguments to call it with.

In this tutorial, you'll learn how to work with various data structures within Gemini Function Calling, including:
    
- Single parameter
- Multiple parameters
- Lists of parameters
- Nested parameters and data structures

## Getting Started


In [ ]:
%pip install --upgrade --quiet google-genai

### Set Google Cloud project information and create client

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
import os

PROJECT_ID = "bdc-trainings"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
# if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
#     PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

from google import genai

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

If above technique does not work, you can try below approach to initialize client using api key

In [ ]:
# API_KEY = "[your-api-key]"  # @param {type: "string", placeholder: "[your-api-key]", isTemplate: true}

# if not API_KEY or API_KEY == "[your-api-key]":
#     raise Exception("You must provide an API key to use Vertex AI in express mode.")

# client = genai.Client(vertexai=True, api_key=API_KEY)

## Code Examples

### Import libraries

In [ ]:
from google.genai.types import FunctionDeclaration, GenerateContentConfig, Tool

### Initialize model


In [ ]:
MODEL_ID = "gemini-2.0-flash"

### Example: Single parameter

Let's say that you want to extract a location from a prompt to help a user navigate to their desired destination.

You can build out a simple schema for a function that takes a single parameter as an input:

In [ ]:
get_destination = FunctionDeclaration(
    name="get_destination",
    description="Get directions to a destination",
    parameters={
        "type": "object",
        "properties": {
            "destination": {
                "type": "string",
                "description": "Destination that the user wants to go to",
            },
        },
    },
)

destination_tool = Tool(
    function_declarations=[get_destination],
)

Now you can send a prompt with a destination, and the model will return structured data with a single key/value pair:

In [ ]:
prompt = "I'd like to travel to Paris"

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(temperature=0, tools=[destination_tool]),
)

response.function_calls

### Example: Multiple parameters

What if you want the function call to return more than one parameter?

You can build out a simple schema for a function that takes multiple parameters as an input:

In [ ]:
get_destination_params = FunctionDeclaration(
    name="get_destination_params",
    description="Get directions to a destination",
    parameters={
        "type": "object",
        "properties": {
            "destination": {
                "type": "string",
                "description": "Destination that the user wants to go to",
            },
            "mode_of_transportation": {
                "type": "string",
                "description": "Mode of transportation to use",
            },
            "departure_time": {
                "type": "string",
                "description": "Time that the user will leave for the destination",
            },
        },
    },
)

destination_tool = Tool(
    function_declarations=[get_destination_params],
)

Now you can send a prompt with a destination, and the model will return structured data with a single key/value pair:

In [ ]:
prompt = "I'd like to travel to Paris by train and leave at 9:00 am"

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(temperature=0, tools=[destination_tool]),
)

response.function_calls

### Example: Lists of parameters

What if you want the function call to return an array or list of parameters within an object?

For example, you might want to call an API to get the geocoded coordinates of several different locations within a single prompt.

In that case, you can build out a schema for a function that takes an array as an input:

In [ ]:
get_multiple_location_coordinates = FunctionDeclaration(
    name="get_location_coordinates",
    description="Get coordinates of multiple locations",
    parameters={
        "type": "object",
        "properties": {
            "locations": {
                "type": "array",
                "description": "A list of locations",
                "items": {
                    "description": "Components of the location",
                    "type": "object",
                    "properties": {
                        "point_of_interest": {
                            "type": "string",
                            "description": "Name or type of point of interest",
                        },
                        "city": {"type": "string", "description": "City"},
                        "country": {"type": "string", "description": "Country"},
                    },
                    "required": [
                        "point_of_interest",
                        "city",
                        "country",
                    ],
                },
            }
        },
    },
)

geocoding_tool = Tool(
    function_declarations=[get_multiple_location_coordinates],
)

Now we'll send a prompt with a few different locations and points of interest:

In [ ]:
prompt = """
    I'd like to get the coordinates for
    the Eiffel tower in Paris,
    the statue of liberty in New York,
    and Port Douglas near the Great Barrier Reef.
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(temperature=0, tools=[geocoding_tool]),
)

response.function_calls

Note that the generative model populated values for all of the parameters for a given location since all three parameters are required.

### Example: Nested parameters and data structures

What if you want the function call to include nested parameters or other complex data structures?

You might want to send a command to create a product listing based on a few sentences that include the product details.

In that case, you can build out a schema for a function that takes nested data structures as an input:

In [ ]:
create_product_listing = FunctionDeclaration(
    name="create_product_listing",
    description="Create a product listing using the details provided by the user.",
    parameters={
        "type": "object",
        "properties": {
            "product": {
                "type": "object",
                "properties": {
                    "name": {"type": "string"},
                    "price": {"type": "number"},
                    "category": {"type": "string"},
                    "description": {"type": "string"},
                },
            }
        },
    },
)

product_listing_tool = Tool(
    function_declarations=[create_product_listing],
)

Now we'll send a prompt with a few different locations and attributes:

In [ ]:
prompt = """Create a listing for noise-canceling headphones for $149.99.
These headphones create a distraction-free environment."""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(temperature=0, tools=[product_listing_tool]),
)

response.function_calls

And you're done! You successfully generated various types of data structures, including a single parameter, multiple parameters, a list of parameters, and nested parameters. Try another notebook to continue exploring other functionality in the Gemini API!